In [1]:
from __future__ import division, print_function, unicode_literals

%matplotlib inline
import matplotlib.pyplot as plt
import pylab as pl
from matplotlib.colors import ListedColormap
import seaborn as sns

import csv
import pandas as pd
import numpy as np  
from io import StringIO
from urllib import request

import warnings

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score 
from sklearn.linear_model import LogisticRegression 
from sklearn import neighbors, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve 
from sklearn.model_selection import ShuffleSplit
from sklearn import linear_model
from sklearn.model_selection import validation_curve
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn import preprocessing

np.random.seed(42)

In [2]:
#Import Data
df= pd.read_csv("/Users/stansonwu/Downloads/ISOM 672/Project/archive/bank-additional-full.csv", header=None, sep=';')
data = df.iloc[1:,:]
data.columns = ['age','job','marital','education','default','housing','loan','contact','month','day of week','duration', 'campaign', 'pdays','previous','poutcome','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed','term deposit subscription']
print(data)

      age          job  marital            education  default housing loan  \
1      56    housemaid  married             basic.4y       no      no   no   
2      57     services  married          high.school  unknown      no   no   
3      37     services  married          high.school       no     yes   no   
4      40       admin.  married             basic.6y       no      no   no   
5      56     services  married          high.school       no      no  yes   
...    ..          ...      ...                  ...      ...     ...  ...   
41184  73      retired  married  professional.course       no     yes   no   
41185  46  blue-collar  married  professional.course       no      no   no   
41186  56      retired  married    university.degree       no     yes   no   
41187  44   technician  married  professional.course       no      no   no   
41188  74      retired  married  professional.course       no     yes   no   

         contact month day of week  ... campaign pdays previous

/Users/stansonwu/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,10,11,12,13,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Data Cleaning

data = data.astype({'age': np.float, 'duration': np.float, 'campaign': np.float, 'pdays': np.float, 'previous': np.float, 'emp.var.rate': np.float, 'cons.price.idx': np.float, 'cons.conf.idx': np.float, 'euribor3m': np.float, 'nr.employed': np.float})

for a in data.columns:
    b = data[a].unique()
    print(a)
    print(b)


age
[56. 57. 37. 40. 45. 59. 41. 24. 25. 29. 35. 54. 46. 50. 39. 30. 55. 49.
 34. 52. 58. 32. 38. 44. 42. 60. 53. 47. 51. 48. 33. 31. 43. 36. 28. 27.
 26. 22. 23. 20. 21. 61. 19. 18. 70. 66. 76. 67. 73. 88. 95. 77. 68. 75.
 63. 80. 62. 65. 72. 82. 64. 71. 69. 78. 85. 79. 83. 81. 74. 17. 87. 91.
 86. 98. 94. 84. 92. 89.]
job
['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
marital
['married' 'single' 'divorced' 'unknown']
education
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
default
['no' 'unknown' 'yes']
housing
['no' 'yes' 'unknown']
loan
['no' 'yes' 'unknown']
contact
['telephone' 'cellular']
month
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
day of week
['mon' 'tue' 'wed' 'thu' 'fri']
duration
[ 261.  149.  226. ... 1246. 1556. 1868.]
campaign
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 1

In [5]:
#Create dummy variables
categorical_data = data.drop('age',axis=1).drop('duration',axis=1).drop('term deposit subscription',axis=1).drop('campaign',axis=1).drop('pdays',axis=1).drop('previous',axis=1).drop('emp.var.rate',axis=1).drop('cons.price.idx',axis=1).drop('cons.conf.idx',axis=1).drop('euribor3m',axis=1).drop('nr.employed',axis=1)

dummy = pd.get_dummies(categorical_data)

       job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  \
1               0                0                 0              1   
2               0                0                 0              0   
3               0                0                 0              0   
4               1                0                 0              0   
5               0                0                 0              0   
...           ...              ...               ...            ...   
41184           0                0                 0              0   
41185           0                1                 0              0   
41186           0                0                 0              0   
41187           0                0                 0              0   
41188           0                0                 0              0   

       job_management  job_retired  job_self-employed  job_services  \
1                   0            0                  0             0   
2    

In [6]:
#Add back numeric variables with dummies variables and turn the target variable into binary variable
dummy['age'] = data['age']
dummy['duration'] = data['duration']
dummy['campaign'] = data['campaign']
dummy['pdays'] = data['pdays']
dummy['previous'] = data['previous']
dummy['emp.var.rate'] = data['emp.var.rate']
dummy['cons.price.idx'] = data['cons.price.idx']
dummy['cons.conf.idx'] = data['cons.conf.idx']
dummy['euribor3m'] = data['euribor3m']
dummy['nr.employed'] = data['nr.employed']

le = preprocessing.LabelEncoder()

le.fit(data['term deposit subscription'])
dummy['term deposit subscription'] = le.transform(data['term deposit subscription'])

print(dummy)

       job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  \
1               0                0                 0              1   
2               0                0                 0              0   
3               0                0                 0              0   
4               1                0                 0              0   
5               0                0                 0              0   
...           ...              ...               ...            ...   
41184           0                0                 0              0   
41185           0                1                 0              0   
41186           0                0                 0              0   
41187           0                0                 0              0   
41188           0                0                 0              0   

       job_management  job_retired  job_self-employed  job_services  \
1                   0            0                  0             0   
2    

In [7]:
#Split data into attributes and target variable
X_dummy = dummy.iloc[:,:-1]
y_dummy = dummy.iloc[:,-1]


In [8]:
#Oversampling
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

print('Original dataset shape %s' % Counter(y_dummy))

ros = RandomOverSampler(random_state=42) 
X_oversampled, y_oversampled = ros.fit_resample(X_dummy, y_dummy)

print('Resampled dataset shape %s' % Counter(y_oversampled))

Original dataset shape Counter({0: 36548, 1: 4640})
Resampled dataset shape Counter({0: 36548, 1: 36548})


In [10]:
#Combine attributes and target variable together after oversampling
final_dummy = pd.concat([X_oversampled, y_oversampled], axis = 1)
print(final_dummy)

       job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  \
0               0                0                 0              1   
1               0                0                 0              0   
2               0                0                 0              0   
3               1                0                 0              0   
4               0                0                 0              0   
...           ...              ...               ...            ...   
73091           0                1                 0              0   
73092           0                1                 0              0   
73093           0                1                 0              0   
73094           1                0                 0              0   
73095           0                0                 0              0   

       job_management  job_retired  job_self-employed  job_services  \
0                   0            0                  0             0   
1    

In [12]:
#Export the modified dataset into a csv file
final_dummy.to_csv('final.csv')
